# Predict new flight positions

## 1. Load the saved model

In [3]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import math

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Context size of our custom model
context_size = 1024

class PositionEmbedding(torch.nn.Module):
    """Token and positioning embedding layer for a sequence."""
    def __init__(self):
        """Init variables and layers."""
        super().__init__()
        
        self.position_emb = torch.nn.Embedding(num_embeddings=context_size, embedding_dim=5)
    
    def forward(self, x):
        """Forward Pass."""
        len_input = x.size()[1]
        positions = torch.arange(start=0, end=len_input, step=1).to(device)
        position_embedding = self.position_emb(positions)
        return x + position_embedding

def create_attention_mask(key_length, query_length, dtype):
    """
    Create a Casual Mask for
    the multi head attention layer.
    """
    i = torch.arange(query_length)[:, None]
    j = torch.arange(key_length)
    mask = i >= j - key_length + query_length
    mask = torch.logical_not(mask)
    mask = mask.to(dtype)
    return mask

class TransformerBlock(torch.nn.Module):
    """Transformer Block Layer."""
    def __init__(self, num_heads, embed_dim, ff_dim, mask_function, dropout_rate=0.1):
        """Init variables and layers."""
        super().__init__()
        self.attn = torch.nn.MultiheadAttention(
          embed_dim=embed_dim,
          num_heads=num_heads,
          batch_first=True,
        )
        self.dropout_1 = torch.nn.Dropout(p=dropout_rate)
        self.layer_norm_1 = torch.nn.LayerNorm(
          normalized_shape=embed_dim, eps=1e-6
        )
        self.ffn_1 = torch.nn.Linear(
          in_features=embed_dim, out_features=ff_dim
        )
        self.ffn_2 = torch.nn.Linear(
          in_features=ff_dim, out_features=embed_dim
        )
        self.dropout_2 = torch.nn.Dropout(p=dropout_rate)
        self.layer_norm_2 = torch.nn.LayerNorm(
          normalized_shape=embed_dim, eps=1e-6
        )
        self.mask_function = mask_function
        self.relu = torch.nn.ReLU()

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """Forward Pass."""
        seq_len = inputs.size()[1]
        mask = self.mask_function(seq_len, seq_len, torch.bool).to(device)
        attention_output, _ = self.attn(
        query=inputs, key=inputs, value=inputs, attn_mask=mask
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.layer_norm_1(inputs + attention_output)
        ffn_1 = self.relu(self.ffn_1(out1))
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        output = self.layer_norm_2(out1 + ffn_output)
        return output

class FlightModel(torch.nn.Module):
  def __init__(self, feed_forward_dim, num_heads):
    """Init Function."""
    super().__init__()
    self.embedding_layer = PositionEmbedding()
    self.transformer = TransformerBlock(
      num_heads=num_heads,
      embed_dim=5,
      ff_dim=feed_forward_dim,
      mask_function=create_attention_mask,
    )
    self.output_layer = torch.nn.Linear(5, 5)

  def forward(self, input_tensor):
    """Forward Pass."""
    embedding = self.embedding_layer(input_tensor)
    transformer_output = self.transformer(embedding)
    output = self.output_layer(transformer_output)
    return output

model = FlightModel(15, 1).to(device)
model.load_state_dict(torch.load("./flight_prediction_model.pt", weights_only=True))
model.eval()

cuda


FlightModel(
  (embedding_layer): PositionEmbedding(
    (position_emb): Embedding(1024, 5)
  )
  (transformer): TransformerBlock(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True)
    )
    (dropout_1): Dropout(p=0.1, inplace=False)
    (layer_norm_1): LayerNorm((5,), eps=1e-06, elementwise_affine=True)
    (ffn_1): Linear(in_features=5, out_features=15, bias=True)
    (ffn_2): Linear(in_features=15, out_features=5, bias=True)
    (dropout_2): Dropout(p=0.1, inplace=False)
    (layer_norm_2): LayerNorm((5,), eps=1e-06, elementwise_affine=True)
    (relu): ReLU()
  )
  (output_layer): Linear(in_features=5, out_features=5, bias=True)
)

## 2. Generate new plane positions

In [33]:
plane_pos_1 = [-33.625946, -70.909181, 14550, 327.9, 22]
plane_pos_2 = [-33.607635, -70.900381, 14225, 324.4, 21.9]
plane_pos = [plane_pos_1, plane_pos_2]
pos_input = torch.tensor(plane_pos)
pos_input = pos_input.view(1, -1, 5)
pos_input = pos_input.to(device)

with torch.no_grad():
    for i in range(10):  
        output = model(pos_input)
        output = torch.squeeze(output)
        output = output[-1]
        output = output.view(1, 1, 5)
        pos_input = torch.cat((pos_input, output), 1)

print(pos_input)


tensor([[[-3.3626e+01, -7.0909e+01,  1.4550e+04,  3.2790e+02,  2.2000e+01],
         [-3.3608e+01, -7.0900e+01,  1.4225e+04,  3.2440e+02,  2.1900e+01],
         [-3.7404e-01, -1.8128e-01,  1.6897e+00,  5.2111e-01, -1.5914e-01],
         [-6.2614e-01, -4.0731e-01,  1.4264e+00,  3.0886e-01, -3.9758e-01],
         [-1.0975e+00, -4.8190e-01,  6.5662e-01,  4.8041e-01,  1.4114e+00],
         [-7.5327e-01, -3.8382e-01,  2.5688e-01,  1.9122e-01,  1.5512e+00],
         [-7.4129e-01, -3.9461e-01,  1.5629e+00,  4.4036e-01,  1.1812e+00],
         [ 7.3108e-02, -3.6159e-02,  1.1081e+00,  1.5331e-01,  4.4667e-01],
         [ 1.2465e-01, -1.1690e-01,  6.7534e-01, -1.1533e-01, -8.1817e-01],
         [-6.2637e-01, -4.0750e-01,  1.4261e+00,  3.0869e-01, -3.9751e-01],
         [-5.7736e-04, -2.4731e-02,  1.2980e+00,  3.0758e-01,  2.8266e-01],
         [-6.2620e-01, -4.0743e-01,  1.4262e+00,  3.0866e-01, -3.9760e-01]]],
       device='cuda:0')
